In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import numpy as np

import cv2 as cv
import xarray as xr

import plotly.express as px 
import plotly.graph_objects as go
import scipy.stats as sst

import my_paths as mp
from src.loaders import get_motif_series

Using CUDA
GPU active: True
GPU used: NVIDIA GeForce GTX 1070


# Manipulation time
of flowers measured from the videos asociated to every visit

what I need to do is a system to relate the visit table to the different videos in a folder

I think that the visit table must have a date with it, including an start hour of aquisition, and an end hour

then we can also add if the visit has an asociated recording and in which camera (future proofing for multiple camera
simultaneous recordings, beecam_1)

### syntetic data
same as before but asociating the 6 analysed recordings with a specific flower

In [3]:
# todo think about the ontology of flowers
#  for example, what to do with flower reward

flower_df = pd.DataFrame([
    {'id_flower': 0, 'color': 'blue', 'sinuosity': 'bend_1',
     'direction': 'right'},
    {'id_flower': 1, 'color': 'white', 'sinuosity': 'bend_1',
     'direction': 'left'},
    {'id_flower': 2, 'color': 'blue', 'sinuosity': 'zig_1',
     'direction': 'right'},
    {'id_flower': 3, 'color': 'white', 'sinuosity': 'zig_1',
     'direction': 'left'}
])

# Creates the synthetic data frame similar to what it might look 
# when acquiring results from experiments.

rng = np.random.default_rng(seed=42)
n_bees = 5
n_events = 300
dates = ["2023-09-13"]
date_repeat = n_events / len(dates)

visit_df = pd.DataFrame(
    {'date': np.repeat(dates, date_repeat).astype('datetime64'),
     'bee': rng.choice(n_bees, n_events).astype('int'),
     'id_flower': rng.choice(
         flower_df.id_flower.unique().tolist(), n_events
     ).astype(int)}
)
# add camera information, this might directly come from the loaded df
id_cam = "beecam_01"
flower_camera = {
    0:id_cam,
    1:None,
    2:None,
    3:None,
}

visit_df['camera'] = visit_df['id_flower'].apply(lambda x: flower_camera[x])
# merge relevant data about flowers
# care with how="left" to preserve the order of the visit_df

working_df = pd.merge(
    left=visit_df, right=flower_df,
    how='left', on='id_flower'
)

working_df.head()

,date,bee,id_flower,camera,color,sinuosity,direction
0,2023-09-13,0,1,None,white,bend_1,left
1,2023-09-13,3,3,None,white,zig_1,left
2,2023-09-13,3,0,beecam_01,blue,bend_1,right
3,2023-09-13,2,3,None,white,zig_1,left
4,2023-09-13,2,2,None,blue,zig_1,right


In [4]:
# access the folder with the camera, get the file names and put them in the right place in a new column
vid_folder = mp.vame_path / "videos"
list(vid_folder.glob("*.mp4"))

[PosixPath('/home/mateo/code/beehaviour/data/bottom_view-Feb20-2024/videos/2024-02-19T13:04:26_1.mp4'),
 PosixPath('/home/mateo/code/beehaviour/data/bottom_view-Feb20-2024/videos/2024-02-19T12:52:24_1.mp4'),
 PosixPath('/home/mateo/code/beehaviour/data/bottom_view-Feb20-2024/videos/2024-02-19T12:32:16_1.mp4'),
 PosixPath('/home/mateo/code/beehaviour/data/bottom_view-Feb20-2024/videos/2024-02-19T12:59:09_2.mp4'),
 PosixPath('/home/mateo/code/beehaviour/data/bottom_view-Feb20-2024/videos/2024-02-19T13:08:00_1.mp4'),
 PosixPath('/home/mateo/code/beehaviour/data/bottom_view-Feb20-2024/videos/2024-02-19T13:09:01_1.mp4'),
 PosixPath('/home/mateo/code/beehaviour/data/bottom_view-Feb20-2024/videos/2024-02-19T12:47:10_1.mp4'),
 PosixPath('/home/mateo/code/beehaviour/data/bottom_view-Feb20-2024/videos/2024-02-19T12:40:17_1.mp4'),
 PosixPath('/home/mateo/code/beehaviour/data/bottom_view-Feb20-2024/videos/2024-02-19T12:32:46_1.mp4'),
 PosixPath('/home/mateo/code/beehaviour/data/bottom_view-Feb20-2

In [5]:
working_df['datum_name'] = np.nan
for date in working_df.date.unique():
    # lists the data with that date and sorts ascending
    # the file extnsion might change...
    # pat = f"{str(date).split('T')[0]}T*.mp4"
    
    # todo delete: temporary pattern for legacy filenames
    day = ''.join(str(date).split('T')[0].split('-'))
    pat = f"*-*-{day}*.mp4"
    
    
    data_names = [file.stem for file in  vid_folder.glob(pat)]
    data_names.sort()
    
    # filters df to get the rows to wich asign videos
    filtered = working_df.query(f"date == '{date}' and camera == '{id_cam}'")
    
    # todo delete: temporally for the synth data matches the table number to the available data/videos
    if True:
        filtered = filtered.iloc[:len(data_names),:]


    if len(data_names) > filtered.shape[0]:
         raise ValueError(f"there is {len(data_names) - filtered.shape[0]} more videos than table registries")
    elif len(data_names) < filtered.shape[0]:
         raise ValueError(f"there is {filtered.shape[0] - len(data_names)} more table registries than videos")
    
    # used the "preserved" index to change the original dataframe
    working_df.loc[filtered.index, 'datum_name'] = data_names


In [6]:
# based on the dataframe now tries to loade a bunch of time series

motif_series = get_motif_series(
working_df.loc[pd.notna(working_df.datum_name),'datum_name'].values
)

In [7]:
motif_series

[]

# split data 

It was unwise to not add a flower name to the video files, therefore I need to add that here somehow.

Fortunately, when I acquired data, the flowers were used sequentially in time, first the left turning, then the right
turning. Asigning directions is as easy as finding the first video where I switched flowers.

In [32]:
videos = np.sort(
    np.asarray([v.stem for v in  vid_folder.glob("*.mp4")])
)

right_start = '2024-02-19T12:38:33_1'

direction = np.empty_like(videos)
direction[:np.argwhere(videos == right_start)[0,0]] = 'zig_02_left'
direction[np.argwhere(videos == right_start)[0,0]:] = 'zig_02_right'

side_df = pd.DataFrame(
    data = [videos, direction] , index= ['datum', 'flower_type']
).T

side_df

,datum,flower_type
0,2024-02-19T12:28:56_1,zig_02_left
1,2024-02-19T12:29:49_1,zig_02_left
2,2024-02-19T12:30:29_1,zig_02_left
3,2024-02-19T12:31:46_1,zig_02_left
4,2024-02-19T12:32:16_1,zig_02_left
5,2024-02-19T12:32:46_1,zig_02_left
6,2024-02-19T12:38:33_1,zig_02_right
7,2024-02-19T12:40:17_1,zig_02_right
8,2024-02-19T12:41:22_1,zig_02_right
9,2024-02-19T12:45:00_1,zig_02_right
